In [3]:
import sys
import time
import config
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extrair_inteiro(texto):
	try:
		i = texto.rindex(' ')
		sem_unidade = texto[:i]

		# Às vezes, esse valor pode iniciar pelo ano...
		i = sem_unidade.find(' ')
		if i >= 0:
			sem_unidade = sem_unidade[(i + 1):]

		sem_virgula = sem_unidade.replace(',', '')

		return int(sem_virgula)
	except:
		return 0

driver = webdriver.Chrome()
driver.get(config.url_inicial)

The chromedriver version (141.0.7390.54) detected in PATH at C:\SeleniumDrivers\chromedriver.exe might not be compatible with the detected chrome version (142.0.7444.176); currently, chromedriver 142.0.7444.175 is recommended for chrome 142.*, so it is advised to delete the driver in PATH and retry


In [4]:
links = WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.ipc-title > a.ipc-title-link-wrapper'))
)

filmes = []

for i in range(0, 20):
	filmes.append({
		"url": links[i].get_attribute("href"),
		"titulo": "",
		"duracao": "",
		"ano": 0,
		"classificacao_etaria": "",
		"nota": 0,
		"posicao": i + 1,
		"generos": [],
		"atores": [],
	})
	

In [5]:
for contador_filme in range(20):
	filme = filmes[contador_filme]
	driver.get(filme["url"])

	time.sleep(1)

	body = driver.find_element(By.TAG_NAME, 'body')

	for i in range(15):
		body.send_keys(Keys.PAGE_DOWN)
		time.sleep(2)

	titulo = WebDriverWait(driver, 1).until(
		EC.presence_of_element_located((By.CSS_SELECTOR, 'span.hero__primary-text'))
	)

	filme["titulo"] = titulo.text.strip()

	meta = WebDriverWait(driver, 1).until(
		EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.sc-af040695-0.iOwuHP > ul > li'))
	)

	if len(meta) > 0:
		filme["ano"] = int(meta[0].text.strip())
		if len(meta) > 1:
			filme["classificacao_etaria"] = meta[1].text.strip()
			if len(meta) > 2:
				filme["duracao"] = meta[2].text.strip()

	try:
		nota = WebDriverWait(driver, 5).until(
			EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.sc-4dc495c1-1.lbQcRY'))
		)

		filme["nota"] = float(nota[0].text.strip().replace(",", "."))
	except:
		filme["nota"] = 0

	for i in range(3):
		try:
			generos = WebDriverWait(driver, 5).until(
				EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li[data-testid="storyline-genres"] div a'))
			)
			break
		except:
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)

	filme["generos"] = []
	filme["atores"] = []

	for genero in generos:
		filme["generos"].append(genero.text.strip())

	atores = WebDriverWait(driver, 1).until(
		EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.ipc-shoveler.title-cast__grid a[data-testid="title-cast-item__actor"]'))
	)

	for ator in atores:
		filme["atores"].append(ator.text.strip())

In [18]:
driver.close()

colocar as funções do arquivo banco.py aqui emabixo

In [ ]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
from config import conexao_banco

engine = create_engine(conexao_banco)

def salvar_filme(filmes):
    ids_filmes = {}#teve que criar diconario pra armazenar os ids pra fazer o relacionamento funcionar

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            resultado = sessao.execute(
                text("""
                    INSERT INTO filme (titulo, duracao, ano, classificacao_etaria)
                    VALUES (:titulo, :duracao, :ano, :classificacao_etaria)
                """),
                filme
            )
            ids_filmes[filme["titulo"]] = resultado.lastrowid

    return ids_filmes


ids_filmes = salvar_filme(filmes)


def salvar_ator(filmes):
    ids_atores = {}
    atores_unicos = set()#isso aqui evita dados duplicados

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            for nome in filme["atores"]:
                if nome not in atores_unicos:
                    atores_unicos.add(nome)

                    resultado = sessao.execute(
                        text("""
                            INSERT INTO ator (nome)
                            VALUES (:nome)
                        """),
                        {"nome": nome}
                    )
                    ids_atores[nome] = resultado.lastrowid

    return ids_atores


ids_atores = salvar_ator(filmes)


def salvar_genero(filmes):
    ids_generos = {}
    generos_unicos = set()

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            for nome in filme["generos"]:
                if nome not in generos_unicos:
                    generos_unicos.add(nome)

                    resultado = sessao.execute(
                        text("""
                            INSERT INTO genero (nome)
                            VALUES (:nome)
                        """),
                        {"nome": nome}
                    )
                    ids_generos[nome] = resultado.lastrowid

    return ids_generos


ids_generos = salvar_genero(filmes)


def salvar_ranking(filmes, ids_filmes):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            sessao.execute(
                text("""
                    INSERT INTO ranking (id_filme, nota, posicao, data)
                    VALUES (:id_filme, :nota, :posicao, CURDATE())
                """),
                {
                    "id_filme": ids_filmes[filme["titulo"]],
                    "nota": filme["nota"],
                    "posicao": filme["posicao"]
                }
            )


salvar_ranking(filmes, ids_filmes)


def salvar_genero_filme(filmes, ids_filmes, ids_generos):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            for nome_genero in filme["generos"]:
                sessao.execute(
                    text("""
                        INSERT INTO genero_filme (id_filme, id_genero)
                        VALUES (:id_filme, :id_genero)
                    """),
                    {
                        "id_filme": ids_filmes[filme["titulo"]],
                        "id_genero": ids_generos[nome_genero]
                    }
                )


salvar_genero_filme(filmes, ids_filmes, ids_generos)

def salvar_elenco(filmes, ids_filmes, ids_atores):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            for nome_ator in filme["atores"]:
                sessao.execute(
                    text("""
                        INSERT INTO elenco (id_ator, id_filme)
                        VALUES (:id_ator, :id_filme)
                    """),
                    {
                        "id_ator": ids_atores[nome_ator],
                        "id_filme": ids_filmes[filme["titulo"]]
                    }
                )


salvar_elenco(filmes, ids_filmes, ids_atores)


IntegrityError: (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry 'Drama' for key 'genero.nome'
[SQL: 
                            INSERT INTO genero (nome)
                            VALUES (%(nome)s)
                        ]
[parameters: {'nome': 'Drama'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [6]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
from config import conexao_banco

engine = create_engine(conexao_banco)

In [7]:
def salvar_filme(filmes):
    ids_filmes = {}

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            # Verifica se já existe
            existente = sessao.execute(
                text("SELECT id FROM filme WHERE titulo = :titulo"),
                {"titulo": filme["titulo"]}
            ).fetchone()

            if existente:
                ids_filmes[filme["titulo"]] = existente[0]
                continue

            # Insere se não existir
            resultado = sessao.execute(
                text("""
                    INSERT INTO filme (titulo, duracao, ano, classificacao_etaria)
                    VALUES (:titulo, :duracao, :ano, :classificacao_etaria)
                """),
                filme
            )
            ids_filmes[filme["titulo"]] = resultado.lastrowid

    return ids_filmes


def salvar_ator(filmes):
    ids_atores = {}

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            for nome in filme["atores"]:

                if nome in ids_atores:
                    continue

                existente = sessao.execute(
                    text("SELECT id FROM ator WHERE nome = :nome"),
                    {"nome": nome}
                ).fetchone()

                if existente:
                    ids_atores[nome] = existente[0]
                    continue

                resultado = sessao.execute(
                    text("INSERT INTO ator (nome) VALUES (:nome)"),
                    {"nome": nome}
                )

                ids_atores[nome] = resultado.lastrowid

    return ids_atores


def salvar_genero(filmes):
    ids_generos = {}

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            for nome in filme["generos"]:

                if nome in ids_generos:
                    continue

                existente = sessao.execute(
                    text("SELECT id FROM genero WHERE nome = :nome"),
                    {"nome": nome}
                ).fetchone()

                if existente:
                    ids_generos[nome] = existente[0]
                    continue

                resultado = sessao.execute(
                    text("INSERT INTO genero (nome) VALUES (:nome)"),
                    {"nome": nome}
                )

                ids_generos[nome] = resultado.lastrowid

    return ids_generos


def salvar_ranking(filmes, ids_filmes):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            sessao.execute(
                text("""
                    INSERT INTO ranking (id_filme, nota, posicao, data)
                    VALUES (:id_filme, :nota, :posicao, CURDATE())
                """),
                {
                    "id_filme": ids_filmes[filme["titulo"]],
                    "nota": filme["nota"],
                    "posicao": filme["posicao"]
                }
            )


def salvar_genero_filme(filmes, ids_filmes, ids_generos):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            id_filme = ids_filmes[filme["titulo"]]

            for nome_genero in filme["generos"]:
                id_genero = ids_generos[nome_genero]

                existente = sessao.execute(
                    text("""
                        SELECT 1 FROM genero_filme
                        WHERE id_filme = :id_filme AND id_genero = :id_genero
                    """),
                    {"id_filme": id_filme, "id_genero": id_genero}
                ).fetchone()

                if existente:
                    continue

                sessao.execute(
                    text("""
                        INSERT INTO genero_filme (id_filme, id_genero)
                        VALUES (:id_filme, :id_genero)
                    """),
                    {"id_filme": id_filme, "id_genero": id_genero}
                )


def salvar_elenco(filmes, ids_filmes, ids_atores):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            id_filme = ids_filmes[filme["titulo"]]

            for nome_ator in filme["atores"]:
                id_ator = ids_atores[nome_ator]

                existente = sessao.execute(
                    text("""
                        SELECT 1 FROM elenco
                        WHERE id_filme = :id_filme AND id_ator = :id_ator
                    """),
                    {"id_filme": id_filme, "id_ator": id_ator}
                ).fetchone()

                if existente:
                    continue

                sessao.execute(
                    text("""
                        INSERT INTO elenco (id_ator, id_filme)
                        VALUES (:id_ator, :id_filme)
                    """),
                    {"id_ator": id_ator, "id_filme": id_filme}
                )




In [ ]:
ids_filmes = salvar_filme(filmes)
ids_atores = salvar_ator(filmes)
ids_generos = salvar_genero(filmes)

salvar_ranking(filmes, ids_filmes)
salvar_genero_filme(filmes, ids_filmes, ids_generos)
salvar_elenco(filmes, ids_filmes, ids_atores)

In [17]:
print(filmes)


[{'url': 'https://www.imdb.com/pt/title/tt1312221/?ref_=chtmvm_t_1', 'titulo': 'Frankenstein', 'duracao': '2 h 29 min', 'ano': 2025, 'classificacao_etaria': '18', 'nota': 7.5, 'posicao': 1, 'generos': ['Drama', 'Fantasia', 'Terror', 'Ficção científica'], 'atores': ['Oscar Isaac', 'Jacob Elordi', 'Christoph Waltz', 'Mia Goth', 'Felix Kammerer', 'Charles Dance', 'David Bradley', 'Lars Mikkelsen', 'Christian Convery', 'Nikolaj Lie Kaas', 'Kyle Gatehouse', 'Lauren Collins', 'Sofia Galasso', 'Joachim Fjelstrup', 'Ralph Ineson', 'Peter Millard', 'Peter MacNeill', 'Burn Gorman']}, {'url': 'https://www.imdb.com/pt/title/tt19847976/?ref_=chtmvm_t_2', 'titulo': 'Wicked: Parte II', 'duracao': '2 h 17 min', 'ano': 2025, 'classificacao_etaria': '10', 'nota': 7.1, 'posicao': 2, 'generos': ['Família', 'Fantasia', 'Musical', 'Romance'], 'atores': ['Cynthia Erivo', 'Ariana Grande', 'Jeff Goldblum', 'Michelle Yeoh', 'Jonathan Bailey', 'Ethan Slater', 'Marissa Bode', 'Colman Domingo', 'Bowen Yang', 'Bron